# Carregar dados históricos da Bovespa em dataframe para análise

In [1]:
import sys
from pandas import Series
import pandas as pd
from datetime import date
pd.__version__

'1.0.5'

In [2]:
import matplotlib as mat
mat.__version__

'3.2.2'

In [3]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [4]:
# Setando opção do Pandas para mostrar todas as linhas
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
# Setando separador de milhar
pd.options.display.float_format = '{:,.2f}'.format

In [6]:
# vr_investimento_padrao = 20000

In [7]:
import time
time.asctime( time.localtime(time.time()) )

'Fri Dec 17 17:39:11 2021'

In [8]:
def carrega_dados():
    layout = [2, 8, 2, 12, 3, 12, 10, 3, 4, 13, 13, 13, 13, 13, 13, 13, 5, 18, 18, 13, 1, 8, 7, 13, 12, 3]
    df = pd.read_fwf("arquivos/COTAHIST_A2021.ZIP", widths=layout, compression='zip',engine='c',
                    names=["TIPREG", "DTEXCH", "CODBDI", "CODNEG", "TPMERC", "NOMRES", "ESPECI", "PRAZOT", "MODREF", "PREABE", "PREMAX", "PREMIN", "PREMED", "PREULT", "PREOFC", "PREOFV", "TOTNEG", "QUATOT", "VOLTOT", "PREEXE", "INDOPC", "DATVEN", "FATCOT", "PTOEXE", "CODISI", "DISMES"])
    df = df.loc[df["CODBDI"] == 2]
    
    df02 = pd.DataFrame({
         "cdAcao":df["CODNEG"]
        ,"dtPregao":pd.to_datetime(df["DTEXCH"], format="%Y%m%d", errors="ignore")
        ,"vrFech":df["PREULT"] / 100
        ,"vrVolume":df["VOLTOT"] / 100
    #    ,"pc_variacao":((df["PREULT"] / df["PREULT"].shift(1)) -1) * 100
        ,"vrMaxDia":df["PREMAX"] / 100
        ,"vrMinDia":df["PREMIN"] / 100
    #    ,"pc_maximo_dia":
    #    ,"pc_minimo_dia":
        ,"vrAbert":df["PREABE"] / 100
    #    ,"pc_abertura":
    #    ,"ic_0_50_pc":
    #    ,"ic_1_00_pc":
    #    ,"ic_1_50_pc":
    #    ,"ic_2_00_pc":
    #    ,"ic_2_50_pc":
    #    ,"ic_3_00_pc":
    #    ,"vr_result_1_00_pc":
    #    ,"vr_result_1_50_pc":
    #    ,"vr_result_2_00_pc":
    #    ,"vr_result_2_50_pc":
    #    ,"vr_result_3_00_pc":
    })
    df02 = df02.sort_values(["cdAcao", "dtPregao"], ascending=True)
    
    df02["pcVar"], df02["pcMaxDia"], df02["pcMinDia"], df02["pcAbert"] = [
         ((df02.vrFech   / df02.vrFech.shift(1)) -1) *100
        ,((df02.vrMaxDia / df02.vrFech.shift(1)) -1) *100
        ,((df02.vrMinDia / df02.vrFech.shift(1)) -1) *100
        ,((df02.vrAbert  / df02.vrFech.shift(1)) -1) *100
        ]
    
    df02["ic05"], df02["ic10"], df02["ic15"], df02["ic20"], df02["ic25"], df02["ic30"] = [
         df02.apply(condicao05, axis=1)
        ,df02.apply(condicao10, axis=1)
        ,df02.apply(condicao15, axis=1)
        ,df02.apply(condicao20, axis=1)
        ,df02.apply(condicao25, axis=1)
        ,df02.apply(condicao30, axis=1)
        ]

    # Filtrando Ação
    # df02.loc[df02["cdAcao"]=="AMBP3"].sort_values(["cdAcao", "dtPregao"], ascending=False)
    
    return df02

In [9]:
def condicao05(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 0.5) else 0

In [10]:
def condicao10(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 1) else 0

In [11]:
def condicao15(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 1.5) else 0

In [12]:
def condicao20(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 2) else 0

In [13]:
def condicao25(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 2.5) else 0

In [14]:
def condicao30(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 3) else 0

In [15]:
# Exclui header e trailer
# df = df.drop(df.index[[0, -1]])

In [16]:
df = carrega_dados()

In [17]:
# df

In [18]:
# Seleciona últimos 25 dias de cada ação
df25d = df.loc[df["dtPregao"]>=(df.dtPregao.drop_duplicates().sort_values(ascending=False).iloc[24])].sort_values(["cdAcao", "dtPregao"], ascending=False)
df15d = df.loc[df["dtPregao"]>=(df.dtPregao.drop_duplicates().sort_values(ascending=False).iloc[14])].sort_values(["cdAcao", "dtPregao"], ascending=False)
df20d = df.loc[df["dtPregao"]>=(df.dtPregao.drop_duplicates().sort_values(ascending=False).iloc[19])].sort_values(["cdAcao", "dtPregao"], ascending=False)

In [19]:
df05 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>0.5).sum()).reset_index(name="0.5%")
df10 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.0).sum()).reset_index(name="resultado")
df15 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.5).sum()).reset_index(name="resultado")
df20 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.0).sum()).reset_index(name="resultado")
df25 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.5).sum()).reset_index(name="resultado")
df30 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>3.0).sum()).reset_index(name="resultado")
dfVol = df25d.groupby("cdAcao")["vrVolume"].agg("mean").reset_index(name="vol")
dfVrFech = df25d.groupby("cdAcao")["vrFech"].agg("mean").reset_index(name="vrFech")

df05_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>0.5).sum()).reset_index(name="0.5%")
df10_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.0).sum()).reset_index(name="resultado")
df15_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.5).sum()).reset_index(name="resultado")
df20_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.0).sum()).reset_index(name="resultado")
df25_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.5).sum()).reset_index(name="resultado")
df30_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>3.0).sum()).reset_index(name="resultado")
dfVol_15d = df15d.groupby("cdAcao")["vrVolume"].agg("mean").reset_index(name="vol")
dfVrFech_15d = df15d.groupby("cdAcao")["vrFech"].agg("mean").reset_index(name="vrFech")

df05_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>0.5).sum()).reset_index(name="0.5%")
df10_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.0).sum()).reset_index(name="resultado")
df15_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.5).sum()).reset_index(name="resultado")
df20_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.0).sum()).reset_index(name="resultado")
df25_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.5).sum()).reset_index(name="resultado")
df30_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>3.0).sum()).reset_index(name="resultado")
dfVol_20d = df20d.groupby("cdAcao")["vrVolume"].agg("mean").reset_index(name="vol")
dfVrFech_20d = df20d.groupby("cdAcao")["vrFech"].agg("mean").reset_index(name="vrFech")

In [20]:
df05["1.0%"], df05["1.5%"], df05["2.0%"], df05["2.5%"], df05["3.0%"] , df05["AvgVol"], df05["AvgVrFech"] = [
    df10["resultado"],df15["resultado"],df20["resultado"],df25["resultado"],df30["resultado"],dfVol["vol"],dfVrFech["vrFech"]
]

df05_15d["1.0%"], df05_15d["1.5%"], df05_15d["2.0%"], df05_15d["2.5%"], df05_15d["3.0%"] , df05_15d["AvgVol"], df05_15d["AvgVrFech"] = [
    df10_15d["resultado"],df15_15d["resultado"],df20_15d["resultado"],df25_15d["resultado"],df30_15d["resultado"],dfVol_15d["vol"],dfVrFech_15d["vrFech"]
]

df05_20d["1.0%"], df05_20d["1.5%"], df05_20d["2.0%"], df05_20d["2.5%"], df05_20d["3.0%"] , df05_20d["AvgVol"], df05_20d["AvgVrFech"] = [
    df10_20d["resultado"],df15_20d["resultado"],df20_20d["resultado"],df25_20d["resultado"],df30_20d["resultado"],dfVol_20d["vol"],dfVrFech_20d["vrFech"]
]

In [21]:
df05     =     df05.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)
df05_15d = df05_15d.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)
df05_20d = df05_20d.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)

In [22]:
pd.set_option('display.max_rows', None)
# df06 = df05.loc[(df05["AvgVol"] > 6000000) & (df05["1.0%"] >= 20) & (df05["AvgVrFech"] > 5)]

In [23]:
df25d['vrVolume'] = df25d['vrVolume'].astype(int)
df25d['vrVolume'] = df25d['vrVolume'].map('{:,}'.format)

### 25 dias, Order by 1%

In [24]:
df05.loc[(df05["AvgVol"] > 6000000) & (df05["1.0%"] >= 24) & (df05["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech
192,BIDI4,24,24,24,24,23,23,"119,437,579.64",12.38
190,BIDI11,24,24,24,24,23,22,"347,739,180.24",37.03
191,BIDI3,24,24,24,23,23,20,"16,938,454.48",12.33
917,POSI3,24,24,24,22,19,17,"40,155,254.52",9.67
247,BPAN4,24,24,24,19,15,13,"92,328,875.04",11.75
1102,TRAD3,24,24,22,22,21,19,"25,322,964.48",6.77
583,GOLL4,24,24,21,19,18,13,"134,195,654.76",16.94
724,LIGT3,25,24,21,18,16,14,"42,925,207.32",11.65


### 20 dias, Order by 1%

In [25]:
df05_20d.loc[(df05_20d["AvgVol"] > 6000000) & (df05_20d["1.0%"] >= 19) & (df05_20d["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech
191,BIDI3,19,19,19,19,19,16,"16,878,147.75",11.88
190,BIDI11,19,19,19,19,18,18,"351,294,354.45",35.63
192,BIDI4,19,19,19,19,18,18,"113,995,062.75",11.89
903,POSI3,19,19,19,17,16,14,"39,339,447.80",9.81
357,CMIN3,19,19,19,17,15,13,"55,709,887.65",6.32
247,BPAN4,19,19,19,16,13,11,"93,223,497.15",11.60
781,MOSI3,19,19,19,15,13,13,"8,508,458.75",9.23
1086,TRAD3,19,19,18,18,17,16,"23,714,266.35",6.87
1010,SEQL3,19,19,18,16,15,13,"38,718,027.90",11.72
574,GOLL4,19,19,18,16,15,11,"140,178,316.80",16.87


### 15 dias, Order by 1%

In [26]:
df05_15d.loc[(df05_15d["AvgVol"] > 6000000) & (df05_15d["1.0%"] >= 14) & (df05_15d["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech
188,BIDI11,14,14,14,14,14,14,"342,554,933.40",35.21
190,BIDI4,14,14,14,14,14,14,"110,645,888.33",11.72
189,BIDI3,14,14,14,14,14,12,"15,843,866.20",11.79
1127,VVEO3,15,14,14,14,12,11,"8,335,363.00",18.67
938,ROMI3,14,14,14,13,12,12,"14,346,407.47",16.94
991,SEQL3,14,14,14,13,12,11,"42,702,474.87",11.86
352,CMIN3,14,14,14,13,11,9,"56,077,580.13",6.40
886,POSI3,14,14,14,12,11,10,"42,460,730.40",9.97
243,BPAN4,14,14,14,12,9,8,"97,074,941.00",11.68
768,MOSI3,14,14,14,11,9,9,"8,657,494.93",9.34


In [27]:
### 25 dias, Order by 0.5%
# df05.loc[(df05["AvgVol"] > 6000000) & (df05["0.5%"] >= 23) & (df05["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)
### 20 dias, Order by 0.5%
# df05_20d.loc[(df05_20d["AvgVol"] > 6000000) & (df05_20d["0.5%"] >= 19) & (df05_20d["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)
### 15 dias, Order by 0.5%
# df05_15d.loc[(df05_15d["AvgVol"] > 6000000) & (df05_15d["0.5%"] >= 15) & (df05_15d["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)

### Consulta ação específica

In [28]:
df25d.loc[(df25d["cdAcao"] == "NUBR33")].replace(0, "")

,cdAcao,dtPregao,vrFech,vrVolume,vrMaxDia,vrMinDia,vrAbert,pcVar,pcMaxDia,pcMinDia,pcAbert,ic05,ic10,ic15,ic20,ic25,ic30
1044573,NUBR33,2021-12-16,9.49,"27,598,675",9.99,9.14,9.61,-0.52,4.72,-4.19,0.73,1,1,1,1,1,1
1042962,NUBR33,2021-12-15,9.54,"37,965,811",9.75,8.93,9.40,3.14,5.41,-3.46,1.62,1,1,1,1,1,1
1041335,NUBR33,2021-12-14,9.25,"71,370,172",10.64,8.91,10.64,-13.15,-0.09,-16.34,-0.09,,,,,,
1039709,NUBR33,2021-12-13,10.65,"136,220,904",11.90,10.35,11.66,-7.39,3.48,-10.00,1.39,1,1,1,1,1,1
1038071,NUBR33,2021-12-10,11.50,"206,686,961",11.73,10.06,10.34,14.54,16.83,0.20,2.99,1,1,1,1,1,1
1036467,NUBR33,2021-12-09,10.04,"51,428,255",10.20,9.40,10.18,-61.97,-61.36,-64.39,-61.44,,,,,,


In [29]:
time.asctime( time.localtime(time.time()) )

'Fri Dec 17 17:40:52 2021'

# Testes com botões e menus

In [30]:
import ipywidgets as widgets

In [31]:
from IPython.display import display, clear_output, Markdown

In [32]:
button = widgets.Button(description='25')
out = widgets.Output()

def on_button_clicked(_):
      # "linking function with output"
      with out:
          # what happens when we press the button
          clear_output()
          print(df05.loc[(df05["AvgVol"] > 6000000) & (df05["1.0%"] >= 25) & (df05["AvgVrFech"] > 5)])

In [33]:
# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([button,out])

In [34]:
menu = widgets.Dropdown(
    options=[25, 20, 15],
    value=25,
    description="Qt Dias:"
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clear_output()
        display(menu)
#       print("changed to %s" % change['new'])
        print(df05.loc[(df05["AvgVol"] > 6000000) & (df05["1.0%"] >= change['new']) & (df05["AvgVrFech"] > 5)])
        
menu.observe(on_change)

display(menu)

Dropdown(description='Qt Dias:', options=(25, 20, 15), value=25)